In [1]:
import pandas as pd
import numpy as np
import collections
import time
from datetime import datetime

# Przygotowanie próbek

In [58]:
train = pd.read_csv('D:\\Dokumenty\\Systemy_rekomendacyjne\\data\\train.csv',
                       sep=',',
                    skiprows=range(1, 7000000),
#                    nrows=100000,
                    index_col=False
                   )
train = train[train["action_type"] == "clickout item"].reset_index().drop(columns=['level_0','index']) 
print(train.shape)
train[:3]

(558765, 12)


,session_id,timestamp,user_id,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,f7d58b4f85653,1541277645,0L46D7KSII49,2,clickout item,1965217,BR,"Formiga, Brazil",mobile,NaN,1965217|6852550|4429658|1668257,131|30|45|54
1,d66c69e569c74,1541277646,574SH1LA1OII,8,clickout item,5067282,US,"East Wenatchee, USA",tablet,NaN,5067282|57083|1005993|354666|10354600|55878|13...,134|152|134|184|83|210|122|180|227|165|242
2,6355c1e21eb0f,1541277646,52JQXBTXCJ98,43,clickout item,2786846,SE,"Istanbul, Turkey",tablet,Hotel|Resort|Breakfast Included|Best Value,5580790|1728901|920947|2515570|37550|2686069|1...,49|42|60|50|59|70|58|47|47|43|60|48|48|51|66|6...


In [59]:
# usunięcie niepotrzebnych kolumn
train = train.drop(columns=['session_id','user_id','action_type','city','current_filters'])
train[:3]

,timestamp,step,reference,platform,device,impressions,prices
0,1541277645,2,1965217,BR,mobile,1965217|6852550|4429658|1668257,131|30|45|54
1,1541277646,8,5067282,US,tablet,5067282|57083|1005993|354666|10354600|55878|13...,134|152|134|184|83|210|122|180|227|165|242
2,1541277646,43,2786846,SE,tablet,5580790|1728901|920947|2515570|37550|2686069|1...,49|42|60|50|59|70|58|47|47|43|60|48|48|51|66|6...


In [60]:
# dodanie godziny z timestampu i usunięcie timestampu
train['hour'] = train.apply(lambda df: datetime.utcfromtimestamp(df['timestamp']).hour, axis=1)
train = train.drop(columns=['timestamp'])
train = train[['reference','step','platform','device','impressions','prices','hour']]
train[:3]

,reference,step,platform,device,impressions,prices,hour
0,1965217,2,BR,mobile,1965217|6852550|4429658|1668257,131|30|45|54,20
1,5067282,8,US,tablet,5067282|57083|1005993|354666|10354600|55878|13...,134|152|134|184|83|210|122|180|227|165|242,20
2,2786846,43,SE,tablet,5580790|1728901|920947|2515570|37550|2686069|1...,49|42|60|50|59|70|58|47|47|43|60|48|48|51|66|6...,20


In [61]:
# rozkład impressions na oddzielne kolumny
df2 = pd.DataFrame(list(train.apply(lambda df: df['impressions'].split('|'), axis=1).values),
             columns=['position_1','position_2','position_3','position_4','position_5',
                      'position_6','position_7','position_8','position_9','position_10',
                      'position_11','position_12','position_13','position_14','position_15',
                      'position_16','position_17','position_18','position_19','position_20',
                      'position_21','position_22','position_23','position_24','position_25'])

train = pd.concat([train, df2], axis=1, sort=False)
train = train.drop(columns=['impressions'])
train[:3]

,reference,step,platform,device,prices,hour,position_1,position_2,position_3,position_4,...,position_16,position_17,position_18,position_19,position_20,position_21,position_22,position_23,position_24,position_25
0,1965217,2,BR,mobile,131|30|45|54,20,1965217,6852550,4429658,1668257,...,None,None,None,None,None,None,None,None,None,None
1,5067282,8,US,tablet,134|152|134|184|83|210|122|180|227|165|242,20,5067282,57083,1005993,354666,...,None,None,None,None,None,None,None,None,None,None
2,2786846,43,SE,tablet,49|42|60|50|59|70|58|47|47|43|60|48|48|51|66|6...,20,5580790,1728901,920947,2515570,...,4241024,4688752,1352598,2786846,80305,80271,84840,3050614,9300664,1648479


In [62]:
# rozkład prices na oddzielne kolumny
df2 = pd.DataFrame(list(train.apply(lambda df: df['prices'].split('|'), axis=1).values),
             columns=['price_position_1','price_position_2','price_position_3','price_position_4','price_position_5',
                      'price_position_6','price_position_7','price_position_8','price_position_9','price_position_10',
                      'price_position_11','price_position_12','price_position_13','price_position_14','price_position_15',
                      'price_position_16','price_position_17','price_position_18','price_position_19','price_position_20',
                      'price_position_21','price_position_22','price_position_23','price_position_24','price_position_25'])

train = pd.concat([train, df2], axis=1, sort=False)
train = train.drop(columns=['prices'])
train[:3]

,reference,step,platform,device,hour,position_1,position_2,position_3,position_4,position_5,...,price_position_16,price_position_17,price_position_18,price_position_19,price_position_20,price_position_21,price_position_22,price_position_23,price_position_24,price_position_25
0,1965217,2,BR,mobile,20,1965217,6852550,4429658,1668257,None,...,None,None,None,None,None,None,None,None,None,None
1,5067282,8,US,tablet,20,5067282,57083,1005993,354666,10354600,...,None,None,None,None,None,None,None,None,None,None
2,2786846,43,SE,tablet,20,5580790,1728901,920947,2515570,37550,...,62,55,65,85,57,55,70,52,52,46


In [63]:
# label encoding: platform, device
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train['platform'])
train['platform'] = le.transform(train['platform'])

le = LabelEncoder()
le.fit(train['device'])
train['device'] = le.transform(train['device'])
train[:3]

,reference,step,platform,device,hour,position_1,position_2,position_3,position_4,position_5,...,price_position_16,price_position_17,price_position_18,price_position_19,price_position_20,price_position_21,price_position_22,price_position_23,price_position_24,price_position_25
0,1965217,2,7,1,20,1965217,6852550,4429658,1668257,None,...,None,None,None,None,None,None,None,None,None,None
1,5067282,8,51,2,20,5067282,57083,1005993,354666,10354600,...,None,None,None,None,None,None,None,None,None,None
2,2786846,43,43,2,20,5580790,1728901,920947,2515570,37550,...,62,55,65,85,57,55,70,52,52,46


In [64]:
# w position_1 do position_25 zamiana None na -1
train.loc[:, 'position_1':'position_25'] = train.loc[:, 'position_1':'position_25'].fillna('-1')
train.loc[:, 'price_position_1':'price_position_25'] = train.loc[:, 'price_position_1':'price_position_25'].fillna('-1')

In [65]:
item_metadata = pd.read_csv('D:\\Dokumenty\\Systemy_rekomendacyjne\\data\\item_metadata_all.csv',
                           sep=','
                           ,usecols=['item_id','clicks_views_ratio','sum_of_properties',
                                   'Satisfactory Rating', 'Shower','Washing Machine', 'Fridge','3 Star',
                                    'Air Conditioning','Good Rating','Car Park' ]
                           )
item_metadata['item_id'] = item_metadata['item_id'].astype(str)
item_metadata.shape

(813442, 11)

In [66]:
# usunięcie wierszy z id, których nie ma w metadata                   
all_ids_train_as_set = set(train.loc[:, 'position_1':'position_25'].values.flatten()).union(set(train['reference'].values)) 
all_ids_train_as_set.discard('-1')
all_ids_item_metadata_as_set = set(item_metadata['item_id'])
missing_ids_in_metadata_as_set = all_ids_train_as_set.difference(all_ids_item_metadata_as_set)

print(len(all_ids_train_as_set))
print(len(all_ids_item_metadata_as_set))
print(len(missing_ids_in_metadata_as_set))

print(train.shape)
train = train.drop(train[(train['reference'].isin(missing_ids_in_metadata_as_set)) |
                (train['position_1'].isin(missing_ids_in_metadata_as_set)) |
                (train['position_2'].isin(missing_ids_in_metadata_as_set)) |
                (train['position_3'].isin(missing_ids_in_metadata_as_set)) |
                (train['position_4'].isin(missing_ids_in_metadata_as_set)) |
                (train['position_5'].isin(missing_ids_in_metadata_as_set)) |
                (train['position_6'].isin(missing_ids_in_metadata_as_set)) |
                (train['position_7'].isin(missing_ids_in_metadata_as_set)) |
                (train['position_8'].isin(missing_ids_in_metadata_as_set)) |
                (train['position_9'].isin(missing_ids_in_metadata_as_set)) |
                (train['position_10'].isin(missing_ids_in_metadata_as_set)) |
                (train['position_11'].isin(missing_ids_in_metadata_as_set)) |
                (train['position_12'].isin(missing_ids_in_metadata_as_set)) |
                (train['position_13'].isin(missing_ids_in_metadata_as_set)) |
                (train['position_14'].isin(missing_ids_in_metadata_as_set)) |
                (train['position_15'].isin(missing_ids_in_metadata_as_set)) |
                (train['position_16'].isin(missing_ids_in_metadata_as_set)) |
                (train['position_17'].isin(missing_ids_in_metadata_as_set)) |
                (train['position_18'].isin(missing_ids_in_metadata_as_set)) |
                (train['position_19'].isin(missing_ids_in_metadata_as_set)) |
                (train['position_20'].isin(missing_ids_in_metadata_as_set)) |
                (train['position_21'].isin(missing_ids_in_metadata_as_set)) |
                (train['position_22'].isin(missing_ids_in_metadata_as_set)) |
                (train['position_23'].isin(missing_ids_in_metadata_as_set)) |
                (train['position_24'].isin(missing_ids_in_metadata_as_set)) |
                (train['position_25'].isin(missing_ids_in_metadata_as_set))].index, axis=0)
print(train.shape)


648831
813442
0
(558765, 55)
(558765, 55)


In [67]:
# dodanie properties dla każdego impressions (position_1-position_25)
start = time.time()
for position in range(1, 26):
    start2 = time.time()
    train = train.merge(item_metadata,
               left_on='position_' + str(position),
               right_on='item_id',
               how='left'
#                ,suffixes=('train','_position_' + str(position))
                       )
    train = train.rename(columns={'clicks_views_ratio':'clicks_views_ratio_position_'+ str(position),
                          'sum_of_properties':'sum_of_properties_position_'+ str(position),
                          'Satisfactory Rating':'Satisfactory Rating_position_'+ str(position),
                          'Shower':'Shower_position_'+ str(position),
                          'Washing Machine':'Washing Machine_position_'+ str(position), 
                          'Fridge':'Fridge_position_'+ str(position),
                          '3 Star':'3 Star_position_'+ str(position),
                          'Air Conditioning':'Air Conditioning_position_'+ str(position),
                          'Good Rating':'Good Rating_position_'+ str(position),
                          'Car Park':'Car Park_position_'+ str(position)})
    end2 = time.time()
    print(end2-start2)
    
end = time.time()
print(end-start)
train[:3]

7.9627745151519775
7.9998087882995605
9.24243426322937
10.35304045677185
12.106982231140137
13.554301023483276
15.333736896514893
17.209447860717773
18.861544132232666
20.733915328979492
23.084938526153564
24.904045820236206
27.46168804168701
29.92224931716919
31.861207485198975
34.855653047561646
37.260981798172
39.71919846534729
43.213651180267334
45.287150144577026
48.60351896286011
51.37092566490173
54.53767681121826
57.359848737716675
60.92672657966614
743.733699798584


,reference,step,platform,device,hour,position_1,position_2,position_3,position_4,position_5,...,clicks_views_ratio_position_25,Shower_position_25,Satisfactory Rating_position_25,Fridge_position_25,Air Conditioning_position_25,Good Rating_position_25,Washing Machine_position_25,3 Star_position_25,Car Park_position_25,sum_of_properties_position_25
0,1965217,2,7,1,20,1965217,6852550,4429658,1668257,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5067282,8,51,2,20,5067282,57083,1005993,354666,10354600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2786846,43,43,2,20,5580790,1728901,920947,2515570,37550,...,0.205882,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,45.0


In [68]:
# usunięcie wszystkich item_id dodanych przez merge
train = train.drop(columns=train.filter(regex=("^item"), axis=1).columns)
train[:3]

,reference,step,platform,device,hour,position_1,position_2,position_3,position_4,position_5,...,clicks_views_ratio_position_25,Shower_position_25,Satisfactory Rating_position_25,Fridge_position_25,Air Conditioning_position_25,Good Rating_position_25,Washing Machine_position_25,3 Star_position_25,Car Park_position_25,sum_of_properties_position_25
0,1965217,2,7,1,20,1965217,6852550,4429658,1668257,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5067282,8,51,2,20,5067282,57083,1005993,354666,10354600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2786846,43,43,2,20,5580790,1728901,920947,2515570,37550,...,0.205882,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,45.0


In [69]:
# label encoding: item_id (reference, impressions)
all_ids_set = set(train.loc[:, 'position_1':'position_25'].values.flatten()).union(set(train['reference'].values)) 
all_ids_list_without_none = list()
for id in all_ids_set:
    if isinstance(id, str):
        all_ids_list_without_none.append(id)
        
le = LabelEncoder()
le.fit(all_ids_list_without_none)

train.loc[:, 'position_1':'position_25'] = train.loc[:, 'position_1':'position_25'].apply(le.transform)
train['reference'] = le.transform(train['reference'])
train[:3]

,reference,step,platform,device,hour,position_1,position_2,position_3,position_4,position_5,...,clicks_views_ratio_position_25,Shower_position_25,Satisfactory Rating_position_25,Fridge_position_25,Air Conditioning_position_25,Good Rating_position_25,Washing Machine_position_25,3 Star_position_25,Car Park_position_25,sum_of_properties_position_25
0,148486,2,7,1,20,148486,451437,318011,126171,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,359940,8,51,2,20,359940,388943,3837,265926,27176,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,215567,43,43,2,20,383499,130831,584605,191400,270449,...,0.205882,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,45.0


In [70]:
# dodanie kolumny difference_mean_price - odchylenie od średniej w grupie wyników
train.loc[:, 'price_position_1':'price_position_25'] = train.loc[:, 'price_position_1':'price_position_25'].astype(float)
train['mean_price'] = train.filter(regex=("^price")).mean(axis=1)
for position in range(1, 26):
    train['difference_mean_price_' + str(position)] = train['price_position_' + str(position)] - train['mean_price']

train = train.drop(columns=['mean_price'])
train[:3]

,reference,step,platform,device,hour,position_1,position_2,position_3,position_4,position_5,...,difference_mean_price_16,difference_mean_price_17,difference_mean_price_18,difference_mean_price_19,difference_mean_price_20,difference_mean_price_21,difference_mean_price_22,difference_mean_price_23,difference_mean_price_24,difference_mean_price_25
0,148486,2,7,1,20,148486,451437,318011,126171,0,...,-10.56,-10.56,-10.56,-10.56,-10.56,-10.56,-10.56,-10.56,-10.56,-10.56
1,359940,8,51,2,20,359940,388943,3837,265926,27176,...,-73.76,-73.76,-73.76,-73.76,-73.76,-73.76,-73.76,-73.76,-73.76,-73.76
2,215567,43,43,2,20,383499,130831,584605,191400,270449,...,6.12,-0.88,9.12,29.12,1.12,-0.88,14.12,-3.88,-3.88,-9.88


In [71]:
# dodanie kolumny odpowiadającej miejscu klikniętego itemu
def add_label_column(df):
    reference = df['reference']
    df['reference'] = -1
    for position in range(1, 26):
        if df['position_' + str(position)] == reference:
            df['reference'] = position
            return df
        
train = train.copy()
train = train.apply(add_label_column, axis=1)
train = train.rename(columns={'reference':'clicked_position'})
train[:3]

,clicked_position,step,platform,device,hour,position_1,position_2,position_3,position_4,position_5,...,difference_mean_price_16,difference_mean_price_17,difference_mean_price_18,difference_mean_price_19,difference_mean_price_20,difference_mean_price_21,difference_mean_price_22,difference_mean_price_23,difference_mean_price_24,difference_mean_price_25
0,1.0,2.0,7.0,1.0,20.0,148486.0,451437.0,318011.0,126171.0,0.0,...,-10.56,-10.56,-10.56,-10.56,-10.56,-10.56,-10.56,-10.56,-10.56,-10.56
1,1.0,8.0,51.0,2.0,20.0,359940.0,388943.0,3837.0,265926.0,27176.0,...,-73.76,-73.76,-73.76,-73.76,-73.76,-73.76,-73.76,-73.76,-73.76,-73.76
2,19.0,43.0,43.0,2.0,20.0,383499.0,130831.0,584605.0,191400.0,270449.0,...,6.12,-0.88,9.12,29.12,1.12,-0.88,14.12,-3.88,-3.88,-9.88


In [72]:
# SPRÓBOWAC -1 NA null i zmienić na liczby niektóre kolumny

#standaryzacja
from sklearn.preprocessing import StandardScaler

list_of_columns = list()
list_of_columns += list(train.filter(regex=("clicks_views_ratio*")).columns)
list_of_columns += list(train.filter(regex=("sum_of_properties*")).columns)
list_of_columns += list(train.filter(regex=("step")).columns)
list_of_columns += list(train.filter(regex=("difference_mean_price*")).columns)

to_std_df = train[list_of_columns]

sc = StandardScaler()
sc.fit(to_std_df)
sc.transform(to_std_df)

train[list_of_columns] = to_std_df
train[:3]

,clicked_position,step,platform,device,hour,position_1,position_2,position_3,position_4,position_5,...,difference_mean_price_16,difference_mean_price_17,difference_mean_price_18,difference_mean_price_19,difference_mean_price_20,difference_mean_price_21,difference_mean_price_22,difference_mean_price_23,difference_mean_price_24,difference_mean_price_25
0,1.0,2.0,7.0,1.0,20.0,148486.0,451437.0,318011.0,126171.0,0.0,...,-10.56,-10.56,-10.56,-10.56,-10.56,-10.56,-10.56,-10.56,-10.56,-10.56
1,1.0,8.0,51.0,2.0,20.0,359940.0,388943.0,3837.0,265926.0,27176.0,...,-73.76,-73.76,-73.76,-73.76,-73.76,-73.76,-73.76,-73.76,-73.76,-73.76
2,19.0,43.0,43.0,2.0,20.0,383499.0,130831.0,584605.0,191400.0,270449.0,...,6.12,-0.88,9.12,29.12,1.12,-0.88,14.12,-3.88,-3.88,-9.88


In [73]:
# usuwanie pustych wierszy (same nulle)
train = train.drop(list(train[train['clicked_position'].isnull()].index))
# train[train['clicked_position'].isnull()]

In [167]:
collections.Counter(train['clicked_position'].values)

Counter({11.0: 11006,
         1.0: 148731,
         2.0: 49173,
         22.0: 4840,
         5.0: 23661,
         6.0: 19163,
         3.0: 34894,
         9.0: 13424,
         4.0: 27902,
         13.0: 8999,
         12.0: 9995,
         17.0: 6642,
         19.0: 5774,
         16.0: 7300,
         10.0: 11959,
         7.0: 16938,
         21.0: 5037,
         14.0: 8367,
         18.0: 6183,
         8.0: 14681,
         25.0: 5214,
         23.0: 4672,
         15.0: 7743,
         24.0: 4629,
         20.0: 5286})

# Nauka

In [74]:
# train = train.drop(columns=train.loc[:,'position_1':'position_25'].columns)
# train[:3]

,clicked_position,step,platform,device,hour,price_position_1,price_position_2,price_position_3,price_position_4,price_position_5,...,difference_mean_price_16,difference_mean_price_17,difference_mean_price_18,difference_mean_price_19,difference_mean_price_20,difference_mean_price_21,difference_mean_price_22,difference_mean_price_23,difference_mean_price_24,difference_mean_price_25
0,1.0,2.0,7.0,1.0,20.0,131.0,30.0,45.0,54.0,-1.0,...,-10.56,-10.56,-10.56,-10.56,-10.56,-10.56,-10.56,-10.56,-10.56,-10.56
1,1.0,8.0,51.0,2.0,20.0,134.0,152.0,134.0,184.0,83.0,...,-73.76,-73.76,-73.76,-73.76,-73.76,-73.76,-73.76,-73.76,-73.76,-73.76
2,19.0,43.0,43.0,2.0,20.0,49.0,42.0,60.0,50.0,59.0,...,6.12,-0.88,9.12,29.12,1.12,-0.88,14.12,-3.88,-3.88,-9.88


In [75]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train.iloc[:,1:], 
                                                    train['clicked_position'],
                                                    test_size=0.2,
                                                    random_state=0)
X_train[:3]

,step,platform,device,hour,price_position_1,price_position_2,price_position_3,price_position_4,price_position_5,price_position_6,...,difference_mean_price_16,difference_mean_price_17,difference_mean_price_18,difference_mean_price_19,difference_mean_price_20,difference_mean_price_21,difference_mean_price_22,difference_mean_price_23,difference_mean_price_24,difference_mean_price_25
275877,3.0,7.0,1.0,21.0,42.0,56.0,64.0,31.0,88.0,57.0,...,3.72,-17.28,-0.28,0.72,-8.28,7.72,-36.28,-11.28,-9.28,-10.28
70957,1.0,27.0,1.0,5.0,13.0,55.0,62.0,16.0,53.0,57.0,...,-6.56,-24.56,-13.56,-11.56,20.44,4.44,-46.56,-46.56,-46.56,-46.56
244723,1.0,43.0,1.0,19.0,134.0,113.0,118.0,85.0,99.0,115.0,...,13.28,-26.72,-9.72,-74.72,-74.72,-74.72,-74.72,-74.72,-74.72,-74.72


In [76]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [77]:
# from keras.models import Sequential
# from keras.layers import Dense, Activation

# model = Sequential()
# model.add(Dense(32, activation='relu', input_dim=54))
# # model.add(Dense(70, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy',f1_m,precision_m, recall_m])
# history = model.fit(X_train.values, y_train.values, epochs=30, batch_size=50)

In [80]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import np_utils

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

# create model
model = Sequential()
model.add(Dense(170, input_dim=304, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(25, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',f1_m,precision_m, recall_m])
history = model.fit(X_train.values, dummy_y, epochs=10, batch_size=100)

Epoch 1/10
446777/446777 [==============================] - 17s 38us/step - loss: nan - accuracy: 0.3222 - f1_m: nan - precision_m: nan - recall_m: nan
Epoch 2/10
446777/446777 [==============================] - 16s 36us/step - loss: nan - accuracy: 0.3223 - f1_m: nan - precision_m: nan - recall_m: nan
Epoch 3/10
446777/446777 [==============================] - 16s 36us/step - loss: nan - accuracy: 0.3223 - f1_m: nan - precision_m: nan - recall_m: nan
Epoch 4/10
446777/446777 [==============================] - 16s 36us/step - loss: nan - accuracy: 0.3223 - f1_m: nan - precision_m: nan - recall_m: nan
Epoch 5/10
446777/446777 [==============================] - 16s 36us/step - loss: nan - accuracy: 0.3223 - f1_m: nan - precision_m: nan - recall_m: nan
Epoch 6/10
446777/446777 [==============================] - 16s 36us/step - loss: nan - accuracy: 0.3223 - f1_m: nan - precision_m: nan - recall_m: nan
Epoch 7/10
446777/446777 [==============================] - 16s 36us/step - loss: nan - 

# XGBOOST

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train.iloc[:,1:], 
                                                    train['clicked_position'],
                                                    test_size=0.2,
                                                    random_state=0)
X_train[:3]

,step,platform,device,hour,position_1,position_2,position_3,position_4,position_5,position_6,...,difference_mean_price_16,difference_mean_price_17,difference_mean_price_18,difference_mean_price_19,difference_mean_price_20,difference_mean_price_21,difference_mean_price_22,difference_mean_price_23,difference_mean_price_24,difference_mean_price_25
10057,127.0,51.0,0.0,1.0,62176.0,68631.0,31161.0,55641.0,75920.0,60715.0,...,78.00,13.00,-6.00,-8.00,78.00,145.00,-81.00,78.00,-82.00,-27.00
451,1.0,31.0,1.0,0.0,135.0,33558.0,130.0,131.0,34533.0,18801.0,...,-17.76,22.24,154.24,-91.76,-77.76,-96.76,-23.76,14.24,-19.76,318.24
1123,2.0,7.0,1.0,0.0,25275.0,28322.0,15147.0,22531.0,39877.0,24123.0,...,-23.36,-23.36,-23.36,-23.36,-23.36,-23.36,-23.36,-23.36,-23.36,-23.36


In [54]:
from xgboost import XGBClassifier
xgbc = XGBClassifier(n_jobs=-1)
xgbc.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=-1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [55]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score, KFold
scores = cross_val_score(xgbc, X_train, y_train, cv=5)

print("Mean cross-validation score: %.2f" % scores.mean())

kfold = KFold(n_splits=10, shuffle=True)
kf_cv_scores = cross_val_score(xgbc, X_train, y_train, cv=kfold )
print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

KeyboardInterrupt: 

In [56]:
y_pred = xgbc.predict(X_test)

In [57]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score

print('balanced_accuracy_score: {0}'.format(balanced_accuracy_score(y_test, y_pred)))
print('accuracy_score: {0}'.format(accuracy_score(y_test, y_pred)))
print('Nieprawidłowo sklasyfikowane próbki: %d' % (y_test != y_pred).sum())
print('classification_report :\n', classification_report(y_test, y_pred))
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(confmat)

balanced_accuracy_score: 0.07655139913057245
accuracy_score: 0.3519561137577595
Nieprawidłowo sklasyfikowane próbki: 22445
classification_report :
               precision    recall  f1-score   support

         1.0       0.36      0.96      0.53     11065
         2.0       0.28      0.16      0.20      3683
         3.0       0.26      0.11      0.15      2597
         4.0       0.29      0.08      0.13      2183
         5.0       0.28      0.06      0.10      1806
         6.0       0.30      0.05      0.08      1372
         7.0       0.35      0.06      0.10      1273
         8.0       0.30      0.03      0.06      1151
         9.0       0.30      0.02      0.05       983
        10.0       0.37      0.04      0.07       915
        11.0       0.30      0.02      0.03       807
        12.0       0.51      0.03      0.05       771
        13.0       0.39      0.03      0.05       635
        14.0       0.38      0.03      0.06       601
        15.0       0.41      0.02      0.

In [179]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(train.dropna().iloc[:,1:], 
                                                    train.dropna()['clicked_position'],
                                                    test_size=0.2,
                                                    random_state=0)

In [187]:
# training a DescisionTreeClassifier 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import confusion_matrix 
dtree_model = DecisionTreeClassifier(max_depth = 200).fit(X_train2, y_train2) 
dtree_predictions = dtree_model.predict(X_test2) 

# creating a confusion matrix 
confusion_matrix(y_test2, dtree_predictions) 

array([[11343,  2376,  1435,  1122,   852,   710,   625,   569,   505,
          397,   397,   337,   333,   303,   266,   235,   207,   213,
          207,   217,   154,   166,   183,   213,   232],
       [ 2140,  1362,   555,   421,   319,   246,   186,   192,   161,
          153,   122,   123,   101,   114,    81,    81,    81,    73,
           77,    75,    65,    57,    68,    55,    84],
       [ 1279,   599,   789,   379,   295,   208,   188,   137,   132,
           91,   116,    92,    62,    67,    71,    73,    59,    43,
           56,    55,    47,    57,    47,    43,    52],
       [ 1032,   418,   383,   572,   272,   166,   159,   139,   111,
           97,    89,    76,    67,    61,    71,    56,    51,    43,
           42,    43,    45,    34,    31,    39,    59],
       [  816,   301,   299,   291,   452,   185,   151,   128,   114,
           80,    93,    60,    67,    54,    52,    39,    59,    34,
           33,    29,    37,    30,    32,    35,    45],


In [188]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score
y_pred=dtree_predictions
print('balanced_accuracy_score: {0}'.format(balanced_accuracy_score(y_test2, y_pred)))
print('accuracy_score: {0}'.format(accuracy_score(y_test2, y_pred)))
print('Nieprawidłowo sklasyfikowane próbki: %d' % (y_test2 != y_pred).sum())
print('classification_report :\n', classification_report(y_test2, y_pred))
confmat = confusion_matrix(y_true=y_test2, y_pred=y_pred)
print(confmat)

balanced_accuracy_score: 0.12081927968024943
accuracy_score: 0.24112313054587367
Nieprawidłowo sklasyfikowane próbki: 54649
classification_report :
               precision    recall  f1-score   support

         1.0       0.51      0.48      0.49     23597
         2.0       0.19      0.19      0.19      6992
         3.0       0.15      0.16      0.15      5037
         4.0       0.13      0.14      0.14      4156
         5.0       0.12      0.13      0.13      3516
         6.0       0.12      0.12      0.12      2866
         7.0       0.11      0.12      0.12      2530
         8.0       0.09      0.10      0.09      2197
         9.0       0.09      0.10      0.10      2032
        10.0       0.10      0.10      0.10      1835
        11.0       0.10      0.11      0.10      1690
        12.0       0.09      0.10      0.10      1589
        13.0       0.10      0.11      0.10      1409
        14.0       0.08      0.08      0.08      1317
        15.0       0.10      0.10      0